# Library

In [57]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)

# Auto reload of library
%reload_ext autoreload
%autoreload 2

from preprocessing_KK import *
from preprocessing_text_KK import * ##
from algorithm_textmining_KK import * ##
from algorithm_machinelearning_KK import *
from algorithm_deeplearning_KK import *
from evaluation_KK import *
from visualization_KK import *

2024-08-12 14:24:46,965	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 14:24:51,722	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 14:24:56,575	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\

# Hyperparameters

In [2]:
TARGET = '의견'
TOKENIZER = 'KeyBERT'

# Data & Preprocessing

In [3]:
# 데이터
## 데이터로딩
df = pd.read_excel(os.path.join(os.getcwd(), 'Data', '시민의견수렴 의견 전처리자료.xlsx'))
## 변수 삭제
colname_delete = ['의제 ID', '사용자 ID', '학교 ID', '환경', '이전 참가 여부', '특이사항', '의견 ID', '수정 시각']
df.drop(columns=colname_delete, inplace=True)
## 무의견 삭제
df.dropna(subset=[TARGET], inplace=True)
## 리인덱싱
df = df.reset_index().iloc[:,1:]
## 기초 전처리
df[TARGET] = df[TARGET].progress_apply(lambda x: text_preprocessor(x, del_number=False, del_bracket_content=False))
## 단어빈도 추출과 변환
wf_soy, waf_soy, \
wf_tf, waf_tf, \
wf_kb, waf_kb, \
sent_keyword = preprocessing_wordfreq(df, colname_target=TARGET, colname_category=None,
                                      num_showkeyword=100, save_local=False, save_name='wordfreq_')
## 결합
df = pd.concat([df, sent_keyword[[each for each in sent_keyword.columns if each.split('_')[1] == TOKENIZER]]], axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 469/469 [00:00<00:00, 1677.13it/s]


training was done. used memory 1.093 Gb
all cohesion probabilities was computed. # words = 501
all branching entropies was computed # words = 1257
all accessor variety was computed # words = 1257
C:/Users/KK/anaconda3/Lib/site-packages/soynlp
cannot access local variable 'f' where it is not associated with a value
cannot access local variable 'f' where it is not associated with a value
before postprocessing 653
_noun_scores_ 225
after postprocessing 144


# Descriptive

In [62]:
# 문장 토큰 길이 분포 확인
plot_histogram_senttoken(df['Token_'+TOKENIZER])    

2024-08-12 14:31:57,596	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 14:32:02,706	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 14:32:08,572	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\

,Length
count,469.0000
mean,7.8742
std,6.1754
min,1.0000
25%,4.0000
50%,6.0000
75%,9.0000
max,43.0000


In [94]:
# 단어 벡티화
df_wordvec, df_wordcorr, df_wordcorrpair, df_sentvec = preprocessing_word2vec(df['Token_'+TOKENIZER])

from sklearn.manifold import TSNE

tsne = TSNE(learning_rate = 100)  
transformed = tsne.fit_transform(np.array([row for row in df_wordvec['word vector']]))

cl = AgglomerativeClustering(10)
cl.fit(transformed);

x_axis_tsne = transformed[:, 0]  
y_axis_tsne = transformed[:, 1]

def plot_tsne_graph(vocabs, x_asix, y_asix):  
    plt.figure(figsize=(15, 15))  
    plt.scatter(x_asix, y_asix, marker = 's', s=150, c=cl.labels_)  
    for i, v in enumerate(vocabs):  
        plt.annotate(v, xy=(x_asix[i], y_asix[i]))

plot_tsne_graph(list(df_wordvec.index), x_axis_tsne, y_axis_tsne)

2024-08-12 15:11:45,317	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 15:11:50,488	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-08-12 15:11:55,293	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\

array([[-0.01158714,  0.01803159,  0.00185557, ..., -0.02491627,
         0.00861297,  0.00297081],
       [-0.01786616,  0.01880977,  0.0025617 , ..., -0.01767569,
        -0.00319574,  0.00214851],
       [-0.00858834,  0.01736094, -0.00979757, ..., -0.01403456,
         0.01478965, -0.00995971],
       ...,
       [ 0.0020048 ,  0.00892342, -0.00877009, ..., -0.00498464,
         0.00040038, -0.00403263],
       [ 0.00764379,  0.00150762, -0.00790589, ..., -0.00817687,
         0.00253129, -0.00258557],
       [-0.00965566,  0.00034161, -0.0021344 , ...,  0.00729543,
         0.00294309, -0.00817752]], dtype=float32)

In [125]:
!pip install bokeh


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [129]:
import pickle

tsne_df = pd.DataFrame(transformed, columns=['x_coord', 'y_coord'])  
tsne_df

tsne_df['word'] = list(df_wordvec.index) 
tsne_df

from bokeh.plotting import figure, show, output_notebook  
from bokeh.models import HoverTool, ColumnDataSource

output_notebook()
plot_data = ColumnDataSource(tsne_df)

tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800, plot_height = 800, active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over

tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle(  
	'x_coord', 'y_coord', source=plot_data,  
	color='red', line_alpha=0.2, fill_alpha=0.1,  
	size=10, hover_line_color='orange'  
)

# adjust visual elements of the plot

tsne_plot.xaxis.visible = False  
tsne_plot.yaxis.visible = False  
tsne_plot.grid.grid_line_color = None  
tsne_plot.outline_line_color = None

# show time!

show(tsne_plot);

Loading BokehJS ...

AttributeError: unexpected attribute 'plot_width' to figure, similar attributes are outer_width, width or min_width

# Modeling

- Latent Dirichlet Allocation (LDA)는 토픽모델링에 이용되는 대표적인 알고리즘

> - 토픽 모델링이 제공하는 토픽은 어떤 주제를 구성하는 단어들인지로 정의되며,
> - 문서 집합에서 이 단어 집합을 찾아 우리의 문서가 어떤 토픽인지 알 수 있음

- LDA 모델을 학습하기 전에 몇 개의 토픽으로 나눌 것인지 먼저 결정해야 한다. 일반적으로 perplexity와 coherence를 함께 고려해 토픽의 개수를 결정한다.

> - perplexity: 모델이 얼마나 잘 예측하는지 나타내는 지표로, 낮은 perplexity 값은 모델이 문서에서 나타나는 단어를 더 잘 예측한다는 것을 의미한다.
> - coherence: 발견된 토픽이 얼마나 의미있는지 나타내는 지표로, 높은 coherence 값은 토픽 내 단어들이 서로 관련성이 높다는 것을 의미한다.

- LDAvis

> (1) Intertopic Distance Map (via multidimensional scaling)
> - 각 토픽이 가진 고차원의 단어들을 PCA로 2차원 축소 시각화
> - 원크기: 토픽의 단어들이 얼마나 포함 및 분포되어 있는지
> - 원거리: 토픽간의 유사성 (2개의 원이 겹친다면 유사한 토픽이라는 의미)
>
> (2) Top-30 Most Relevant Terms for Topic
> - 토픽 별로 가장 자주 등장하는 단어들(람디=1)과 토픽 간 차이가 많이나는 단어들(람다=0)을 종합적으로 고려 

In [120]:
# 주체 추출
# topic_kw, doc_topic, model = modeling_LDA(df['Token_'+TOKENIZER], opt_numtopics=False, 
#                                           num_topics=2, num_topicwords=10, num_topicsamples=10,
#                                           save_local=True, save_name='Topics_byLDA.csv')
topic_kw, doc_topic, model = modeling_LDA(df[TARGET], opt_numtopics=False, 
                                          num_topics=6, num_topicwords=10, num_topicsamples=10,
                                          save_local=True, save_name='Topics_byLDA.csv')
display(topic_kw, doc_topic)

# 토픽별 wordcloud (다른 카테고리 응용 추가)



[Kss]: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


,Related Keywords,Number of Sentences,Percentage
Topic 1,"'아동(23.5%)', '있는(14.7%)', '교육(9.8%)', '학교(9.8%)', '권리(7.8%)', '가족(6.9%)', '환경(6.9%)', '한다(6.9%)'...",94,0.2004
Topic 2,"'놀이(21.6%)', '있는(18.2%)', '시설(11.4%)', '의견을(9.1%)', '아동의(9.1%)', '아동들이(8.0%)', '아동이(5.7%)', '위한(...",60,0.1279
Topic 3,"'있는(20.5%)', '아동들이(16.9%)', '아동이(9.6%)', '위한(8.4%)', '환경(8.4%)', '한다(7.2%)', '있도록(7.2%)', '놀이(7....",73,0.1557
Topic 4,"'교육(28.4%)', '활동(8.8%)', '아동이(8.8%)', '가족(7.8%)', '프로그램(7.8%)', '한다(7.8%)', '가족과(7.8%)', '있는(7.8...",83,0.1770
Topic 5,"'지원(15.1%)', '아동(15.1%)', '한다(12.3%)', '환경(9.6%)', '있는(8.2%)', '체험(8.2%)', '학교(8.2%)', '교육(8.2%)...",77,0.1642
Topic 6,"'놀이(28.6%)', '안전(13.2%)', '합니다(9.9%)', '아동(8.8%)', '의견을(7.7%)', '있는(7.7%)', '아동의(7.7%)', '주기(5.5...",82,0.1748


,의견,Related Topic,Weight,Each Topic Weights
0,돈 지원금 지급,6,0.4824,"[(0, 0.04185477), (1, 0.35003775), (2, 0.0417701), (3, 0.041670702), (4, 0.04231225), (5, 0.4823..."
1,가족 동반 체험,5,0.7901,"[(0, 0.042288058), (1, 0.041698594), (2, 0.041756675), (3, 0.04240749), (4, 0.7901332), (5, 0.04..."
2,생활비 지원 생리대 화장지 필수 용품 지원,2,0.8955,"[(0, 0.020868378), (1, 0.89546853), (2, 0.02086884), (3, 0.020876529), (4, 0.021059468), (5, 0.0..."
3,화목한 설문 조사,3,0.6429,"[(0, 0.18996921), (1, 0.04190829), (2, 0.64285105), (3, 0.041671544), (4, 0.041730445), (5, 0.04..."
4,가족과 즐거운 대화를 한다 가족과 지내는 방법을 알러준다,4,0.9069,"[(0, 0.018602517), (1, 0.01855226), (2, 0.018686838), (3, 0.90690684), (4, 0.0187107), (5, 0.018..."
...,...,...,...,...
464,건강한 음식 위생관리,6,0.7911,"[(0, 0.04167147), (1, 0.041787755), (2, 0.04193278), (3, 0.041736625), (4, 0.041774947), (5, 0.7..."
465,건강한 음식을 지원하고 마음건강을 위한 설문 조사 건강한 음식 지원,3,0.9237,"[(0, 0.015206854), (1, 0.015264282), (2, 0.92372036), (3, 0.015173897), (4, 0.015292478), (5, 0...."
466,친구관계 전자기기 사용 줄이기 주기적으로 학생들의 미디어 건강 프로그램을 하기,4,0.9240,"[(0, 0.015193975), (1, 0.0151807), (2, 0.0152625125), (3, 0.92399836), (4, 0.015191797), (5, 0.0..."
467,학교 밖 아동 관리 건강한 음식 지원,5,0.8950,"[(0, 0.020982578), (1, 0.020940894), (2, 0.021013526), (3, 0.020893207), (4, 0.89495736), (5, 0...."
